In [2]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
from thesis.utils.pytorch import train

In [3]:
# Define data transforms
# MobileNetV2 expects 224x224 input images and specific normalization
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(256),
        transforms.RandomResizedCrop(224),          # phóng to rồi crop ngẫu nhiên vùng 224x224
        transforms.RandomHorizontalFlip(p=0.5),     # lật ngang ảnh ngẫu nhiên
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # thay đổi màu sắc
        transforms.RandomRotation(degrees=15),      # xoay ngẫu nhiên ±15 độ
        transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),  # dịch ảnh
        transforms.RandomPerspective(distortion_scale=0.2, p=0.5), # hiệu ứng phối cảnh
        transforms.ToTensor(),                      # chuyển thành tensor
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([ # Often same as validation for consistent evaluation
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


# base_dir = "/Users/anand/Desktop/1mg/repos/thesis/thesis/data/PlantDoc-Dataset"
base_dir = "/Users/anand/Desktop/temp/Data for Identification of Plant Leaf Diseases Using a 9-layer Deep Convolutional Neural Network/Plant_leave_diseases_dataset_without_augmentation"
base_dataset = datasets.ImageFolder(base_dir)

total_size = len(base_dataset)
print('Total size:', total_size)
train_size = int(0.8 * total_size)
val_size = int(0.1 * total_size)
test_size = total_size - train_size - val_size

print('Train size:', train_size)
print('Val size:', val_size)
print('Test size:', test_size)

train_data, val_data, test_data = random_split(
    base_dataset, [train_size, val_size, test_size],
    generator=torch.Generator().manual_seed(42)
)

train_data.dataset.transform = data_transforms["train"]
val_data.dataset.transform = data_transforms["val"]
test_data.dataset.transform = data_transforms["test"]

batch_size = 64
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, pin_memory=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_data, shuffle=True, pin_memory=True)


Total size: 55448
Train size: 44358
Val size: 5544
Test size: 5546


In [34]:
from collections import Counter

# These give you the label for each sample in the subset
train_labels = [label for _, label in train_data]
val_labels = [label for _, label in val_data]
test_labels = [label for _, label in test_data]

train_counters = Counter(train_labels)
val_counters =  Counter(val_labels)
test_counters = Counter(test_labels)

In [35]:
x = list(train_counters.keys())
x.sort()
print(x)

y = list(val_counters.keys())
y.sort()
print(y)

z = list(test_counters.keys())
z.sort()
print(z)


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38]


In [5]:
# device = torch.device("mps" if torch.mps.is_available() else "cpu")
device = torch.device("cuda" if torch.cuda.is_available() else "mps")
print("Using device:", device)

Using device: mps


In [ ]:
"""Quantisation step"""

import torch
import torchvision.models

# Check what backends are available
print("Available quantization engines:", torch.backends.quantized.supported_engines)

# Use the first available backend
if torch.backends.quantized.supported_engines:
    torch.backends.quantized.engine = torch.backends.quantized.supported_engines[0]
    print(f"Using quantization engine: {torch.backends.quantized.engine}")
else:
    print("No quantization engines available")

model = torchvision.models.mobilenet_v3_large(weights="DEFAULT")
model = model.to("cpu")
model.eval()

model = torch.quantization.quantize_dynamic(
    model, {torch.nn.Linear}, dtype=torch.qint8
)

Available quantization engines: ['qnnpack', 'none']
Using quantization engine: qnnpack


In [7]:
import torchvision

model = torchvision.models.mobilenet_v3_large(weights="DEFAULT")

# Get number of classes
num_classes = len(base_dataset.classes) 
print("Num classes: ", num_classes)

# Check classifier structure and get input features
print("Classifier:", model.classifier)

# For MobileNetV3, the final layer is usually at index -1 or index 3
# Let's try the last layer
final_layer = model.classifier[-1]  # or try model.classifier[3]
in_features = final_layer.in_features
print("No of features: ", in_features)

# Replace the final layer
model.classifier[-1] = nn.Linear(in_features, num_classes)

# Move to device
model = model.to(device)

# Freeze feature extraction layers
for param in model.features.parameters():
    param.requires_grad = False

# Setup loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

Num classes:  39
Classifier: Sequential(
  (0): Linear(in_features=960, out_features=1280, bias=True)
  (1): Hardswish()
  (2): Dropout(p=0.2, inplace=True)
  (3): Linear(in_features=1280, out_features=1000, bias=True)
)
No of features:  1280


In [38]:
num_epochs = 30
print("device", device)
train("mobile_net_v3", num_epochs, model, train_data, train_loader, test_data, test_loader, criterion, optimizer, device)

device mps
============================== Epoch 1:


Train:   0%|          | 0/694 [00:00<?, ?it/s]/Users/anand/Desktop/1mg/repos/thesis/.venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
Train: 100%|██████████| 694/694 [01:45<00:00,  6.56it/s]


===Train:	|	Accuracy: 0.9047	|	Loss: 0.3294


Val: 100%|██████████| 5546/5546 [00:54<00:00, 101.74it/s]


===Valid:	|	Accuracy: 0.9522	|	Loss: 0.1361



============================== Epoch 2:


...

===Valid:	|	Accuracy: 0.9749	|	Loss: 0.1483



============================== Epoch 30:


Train: 100%|██████████| 694/694 [01:45<00:00,  6.59it/s]


===Train:	|	Accuracy: 0.9934	|	Loss: 0.0251


Val: 100%|██████████| 5546/5546 [00:54<00:00, 102.68it/s]


===Valid:	|	Accuracy: 0.9701	|	Loss: 0.1861



Best checkpoinpt: 27
Train Accuracy: 0.9949501780963975	|	Train Loss: 0.016309875207539112
Test Accuracy: 0.9675441759826903	|	Test Loss: 0.17686573445059442
✅ Accuracy: 0.9701 (5380/5546)


In [ ]:
from thesis.utils.pytorch import train_with_plots


num_epochs = 30
print("device", device)
train_with_plots("mobile_net_v2", num_epochs, model, train_data, train_loader, test_data, test_loader, criterion, optimizer, device)

device mps
============================== Epoch 1/30


Train:   0%|          | 0/694 [00:00<?, ?it/s]/Users/anand/Desktop/1mg/repos/thesis/.venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
Val: 100%|██████████| 5546/5546 [00:55<00:00, 99.58it/s] 


Train:	Accuracy: 0.9021	Loss: 0.3339
Valid:	Accuracy: 0.9522	Loss: 0.1465
Time taken: 162.37 seconds

============================== Epoch 2/30


Val: 100%|██████████| 5546/5546 [00:54<00:00, 102.10it/s]


Train:	Accuracy: 0.9618	Loss: 0.1181
Valid:	Accuracy: 0.9656	Loss: 0.1033
Time taken: 163.67 seconds

============================== Epoch 3/30


...

Val: 100%|██████████| 5546/5546 [00:53<00:00, 103.81it/s]


Train:	Accuracy: 0.9940	Loss: 0.0212
Valid:	Accuracy: 0.9683	Loss: 0.1852
Time taken: 164.62 seconds

============================== Epoch 28/30


Val: 100%|██████████| 5546/5546 [00:53<00:00, 104.24it/s]


Train:	Accuracy: 0.9947	Loss: 0.0197
Valid:	Accuracy: 0.9702	Loss: 0.2052
Time taken: 161.53 seconds

============================== Epoch 29/30


Train:  21%|██▏       | 148/694 [00:24<01:32,  5.90it/s]